In [1]:
%%capture
!pip install pygame
!pip install pygad

!git clone https://github.com/karinemiras/evoman_framework.git tmp
!cp -r /kaggle/working/tmp/* /kaggle/working/
!rm -R /kaggle/working/tmp

In [2]:
import sys, os
import numpy as np
import pandas as pd

import random

from evoman.environment import Environment
from evoman.controller import Controller
from demo_controller import player_controller
from deap import base, creator, tools, algorithms
import multiprocessing
import optuna

import seaborn as sns
import matplotlib.pyplot as plt

pygame 2.5.2 (SDL 2.28.2, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
def parameter_count(hidden_neurons):
    if hidden_neurons>0:
        n_w = (20*hidden_neurons) + (hidden_neurons*5)
        n_b = hidden_neurons + 5
        return n_w + n_b
    else:
        return (20*5)+5
    
def fitness_function(args):
    solution, train_enemies = args
    gains = []
    for enemy in train_enemies:
        env.update_parameter("enemies",[enemy])
        res = env.play(np.array(solution))
        gains.append(res[1]-res[2])
    sum_gain = sum(gains)
    defeated = sum([1 if g>=0 else 0 for g in gains])+1
    if sum_gain<0:
        final = (sum_gain/len(train_enemies))/defeated
    else:
        final = (sum_gain/len(train_enemies))*defeated
    return (final,)

"""
def fitness_function(solution):
    defeated = 0
    for enemy in range(1,9):
        env.update_parameter("enemies",[enemy])
        res = env.play(np.array(solution))
        if res[2]<=0:
            defeated += 1
    return (defeated,)
"""

'\ndef fitness_function(solution):\n    defeated = 0\n    for enemy in range(1,9):\n        env.update_parameter("enemies",[enemy])\n        res = env.play(np.array(solution))\n        if res[2]<=0:\n            defeated += 1\n    return (defeated,)\n'

In [5]:
def cxWholeAritmetic(parent1,parent2,alpha):
    child1 = (alpha*np.array(parent1))+((1-alpha)*np.array(parent2))
    child2 = (alpha*np.array(parent2))+((1-alpha)*np.array(parent1))
    return creator.Individual(child1),creator.Individual(child2)

def cxBlend(parent1,parent2,alpha):
    gamma = (1+2*alpha)*np.random.uniform(size=(len(parent1,))) - alpha
    child1 = (gamma*np.array(parent1))+((1-gamma)*np.array(parent2))
    child2 = (gamma*np.array(parent2))+((1-gamma)*np.array(parent1))
    return creator.Individual(child1),creator.Individual(child2)

In [6]:
def mutGaussian(ind,sigma,indpb):
    mutation = [np.random.normal(0,sigma) if np.random.uniform()<indpb else 0 for i in range(len(ind))]
    return creator.Individual(np.array(ind) + np.array(mutation))

def mutGaussianAdaptive(ind,sigma,tau,indpb):
    new_sigma = sigma * np.exp(np.random.normal(0,tau,size=(len(ind),)))
    new_sigma[new_sigma<0.1] = 0.1
    mutation = [np.random.normal(0,s) if np.random.uniform()<indpb else 0 for s in new_sigma]
    return creator.Individual(np.array(ind) + np.array(mutation))

In [7]:
def genetic_algorithm(train_enemies,initial_range,pop_size,n_offspring,tournsize_parent,cxpb,cx_type,
                      alpha,mutpb,mut_type,sigma,indpb,tau,tournsize_survival,ngen,verbose):
    
    global individual_size,env,creator,stats

    
    toolbox = base.Toolbox()
    toolbox.register("weight_bin", np.random.uniform,-initial_range,initial_range)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.weight_bin, n=individual_size)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("evaluate", fitness_function)

    pop = toolbox.population(n=pop_size)
    hof = {'best_solution':-np.inf,'max_value':-np.inf}

    with multiprocessing.Pool() as pool:
        # evaluate initial population
        for ind, fit in zip(pop, pool.map(toolbox.evaluate,zip(pop,[train_enemies for i in range(len(pop))]))):
            ind.fitness.values = fit

        # record best solution
        for ind in pop:
            if ind.fitness.values[0]>hof['max_value']:
                hof['best_solution'] = ind
                hof['max_value'] = ind.fitness.values[0]

        logbook = tools.Logbook()
        logbook.header = ['gen', 'nevals'] + (stats.fields if stats else []) + ['max_hof']
        record = stats.compile(pop) if stats else {}

        logbook.record(gen=0, nevals=len(pop), **record, max_hof=hof['max_value'])
        if verbose==1:
            print(logbook.stream)

        for g in range(ngen):
            # select parents
            parents = tools.selTournament(individuals=pop,k=n_offspring*pop_size,tournsize=tournsize_parent)

            # create offspring
            offspring = []
            for child1, child2 in zip(parents[::2], parents[1::2]):
                if np.random.uniform() < cxpb:
                    if cx_type=="whole_aritmetic":
                        if alpha=="random":
                            alpha=np.random.uniform()
                            child1,child2 = cxWholeAritmetic(child1, child2, alpha)
                        else:
                            child1,child2 = cxWholeAritmetic(child1, child2, alpha)
                    elif cx_type=="blend":
                        if alpha=="random":
                            alpha=np.random.uniform()
                            child1,child2 = cxBlend(child1, child2, alpha)
                        else:
                            child1,child2 = cxBlend(child1, child2, alpha)
                offspring.append(child1)
                offspring.append(child2)

            # apply mutation to offspring
            for i in range(len(offspring)):
                if np.random.uniform() < mutpb:
                    if mut_type=="normal":
                        if indpb=="random":
                            indpb=np.random.uniform()
                            offspring[i] = mutGaussian(offspring[i],sigma,indpb)
                        else:
                            offspring[i] = mutGaussian(offspring[i],sigma,indpb)
                    elif mut_type=="adaptive":
                        if indpb=="random":
                            indpb=np.random.uniform()
                            offspring[i] = mutGaussianAdaptive(offspring[i],sigma,indpb,tau)
                        else:
                            offspring[i] = mutGaussianAdaptive(offspring[i],sigma,indpb,tau)       

            # evaluate offspring
            invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
            for ind, fit in zip(invalid_ind, pool.map(toolbox.evaluate,zip(invalid_ind,[train_enemies for i in range(len(invalid_ind))]))):
                    ind.fitness.values = fit

            # record best solution
            for ind in offspring:
                if ind.fitness.values[0]>hof['max_value']:
                    hof['best_solution'] = ind
                    hof['max_value'] = ind.fitness.values[0]

            # select the next generation of the population
            pop = tools.selTournament(individuals=offspring,k=pop_size,tournsize=tournsize_survival)

            record = stats.compile(pop) if stats else {}
            logbook.record(gen=g+1, nevals=len(invalid_ind), **record, max_hof=hof['max_value'])
            if verbose==1:
                print(logbook.stream)
        pool.close()
        pool.join()

    return hof,logbook

In [8]:
hidden_neurons = 10
individual_size = parameter_count(hidden_neurons)

headless = True
if headless:
    os.environ["SDL_VIDEODRIVER"] = "dummy"

env = Environment(playermode="ai",
                  player_controller=player_controller(hidden_neurons),
                  speed="fastest",
                  enemymode="static",
                  level=2,
                  visuals=False,
                  logs="off")

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("Mean", np.mean)
stats.register("Max", np.max)
stats.register("Std", np.std)

ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5181:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5181:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5181:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5704:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2666:(snd_pcm_open_noupdate) Unknown PCM default


In [9]:
def objective(trial):
    
    params = {'train_enemies':trial.suggest_categorical("train_enemies",[[7,8],[2,5,6],[1,2,6,7]]),
              'initial_range':trial.suggest_int("initial_range", 1,3,step=1),
              'pop_size':trial.suggest_int("pop_size", 50,50,step=50),
              'n_offspring':trial.suggest_int("n_offspring", 1,5,step=1),
              'tournsize_parent':trial.suggest_int("tournsize_parent", 1,10,step=1),
              'cxpb':trial.suggest_float("cxpb", 0.1, 1., step=0.1),
              'cx_type':trial.suggest_categorical("cx_type",["whole_aritmetic","blend"]),
              'alpha':trial.suggest_categorical("alpha",np.arange(0.1,1.1,0.1).tolist()+["random"]),
              'mutpb':trial.suggest_float("mutpb", 0.1, 1., step=0.1),
              'mut_type':trial.suggest_categorical("mut_type",["normal","adaptive"]),
              'sigma':trial.suggest_float("sigma", 0.5, 3, step=0.5),
              'indpb':trial.suggest_categorical("indpb",np.arange(0.1,1.1,0.1).tolist()+["random"]),
              'tau':trial.suggest_float("tau", 0.1, 1., step=0.1),
              'tournsize_survival':trial.suggest_int("tournsize_survival", 1,10,step=1),
              'ngen':trial.suggest_int("ngen", 30,30,step=20),
              'verbose':trial.suggest_int("verbose", 1,1,step=1)}
    
    global hof_tuning
    iteration_logs = []
    for i in range(3):
        hof,logs = genetic_algorithm(**params)
        logs_pd = pd.DataFrame(logs)
        iteration_logs.append(logs_pd)
        hof_tuning.append(hof)
    
    return pd.concat(iteration_logs)['Max'].mean()


In [10]:
hof_tuning = []

study = optuna.create_study(study_name="GA", direction="maximize")

study.optimize(objective, n_trials=500)

[I 2023-10-15 00:23:21,266] A new study created in memory with name: GA
/opt/conda/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [7, 8] which is of type list.
  warnings.warn(message)
/opt/conda/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [2, 5, 6] which is of type list.
  warnings.warn(message)
/opt/conda/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [1, 2, 6, 7] which is of type list.
  warnings.warn(message)


gen	nevals	Mean   	Max 	Std    	max_hof
0  	50    	-81.817	27.8	30.9501	27.8   
1  	50    	-47.09 	25.6	42.4528	27.8   
2  	50    	12.543 	98.4	34.9574	98.4   
3  	50    	67.151 	136.2	50.4737	136.2  
4  	50    	71.277 	146.1	55.7058	146.1  
5  	50    	75.964 	131.7	31.4612	146.1  
6  	50    	110.716	140.7	35.8276	146.1  
7  	50    	109.346	147.9	38.2607	147.9  
8  	50    	107.354	146.1	36.3864	147.9  
9  	50    	121.112	146.1	22.6489	147.9  
10 	50    	128.254	143.4	23.4139	147.9  
11 	50    	123.654	148.8	17.4993	148.8  
12 	50    	137.208	155.1	16.5195	155.1  
13 	50    	144.12 	175.8	22.1766	175.8  
14 	50    	127.322	172.2	44.5504	175.8  
15 	50    	139.71 	180.3	27.1151	180.3  
16 	50    	123.778	159.6	27.5084	180.3  
17 	50    	137.232	172.2	30.6455	180.3  
18 	50    	161.868	181.2	19.6754	181.2  
19 	50    	154.816	201  	37.9853	201    
20 	50    	158.286	174  	20.726 	201    
21 	50    	168.744	180.3	11.3776	201    
22 	50    	151.476	182.1	29.931 	201    
23 	50    	157.926	1

[I 2023-10-15 00:26:03,486] Trial 0 finished with value: 157.69462365591482 and parameters: {'train_enemies': [7, 8], 'initial_range': 2, 'pop_size': 50, 'n_offspring': 1, 'tournsize_parent': 4, 'cxpb': 0.30000000000000004, 'cx_type': 'blend', 'alpha': 0.4, 'mutpb': 1.0, 'mut_type': 'adaptive', 'sigma': 3.0, 'indpb': 0.2, 'tau': 0.7000000000000001, 'tournsize_survival': 6, 'ngen': 30, 'verbose': 1}. Best is trial 0 with value: 157.69462365591482.


30 	50    	161.634	183  	18.5573	194.7  
gen	nevals	Mean   	Max  	Std    	max_hof
0  	50    	-88.558	-2.55	24.3773	-2.55  
1  	142   	-37.926	19.8 	36.7615	19.8   
2  	144   	13.55  	105.6	40.0104	105.6  
3  	142   	28.3   	119.1	52.9062	119.1  
4  	139   	38.191 	158.7	58.347 	158.7  
5  	140   	42.726 	158.7	68.8091	158.7  
6  	140   	61.532 	158.7	75.4443	158.7  
7  	142   	73.683 	158.7	59.6852	158.7  
8  	139   	89.513 	158.7	58.6599	158.7  
9  	142   	86.893 	158.7	63.1228	158.7  
10 	140   	87.31  	158.7	61.3037	158.7  
11 	140   	73.633 	158.7	68.0981	158.7  
12 	136   	84.568 	158.7	64.486 	158.7  
13 	139   	93.618 	158.7	58.9298	158.7  
14 	138   	85.77  	158.7	70.7406	158.7  
15 	139   	83.608 	158.7	64.5832	158.7  
16 	139   	81.744 	158.7	69.2934	158.7  
17 	142   	73.454 	158.7	60.7297	158.7  
18 	136   	106.73 	175.8	65.4768	175.8  
19 	134   	86.461 	175.8	71.7841	175.8  
20 	141   	56.456 	175.8	55.9861	175.8  
21 	145   	75.67  	175.8	56.3032	175.8  
22 	141   	97.84

[I 2023-10-15 00:33:22,825] Trial 1 finished with value: 145.8747311827965 and parameters: {'train_enemies': [7, 8], 'initial_range': 1, 'pop_size': 50, 'n_offspring': 3, 'tournsize_parent': 6, 'cxpb': 0.30000000000000004, 'cx_type': 'blend', 'alpha': 1.0, 'mutpb': 0.9, 'mut_type': 'normal', 'sigma': 1.0, 'indpb': 0.7000000000000001, 'tau': 0.7000000000000001, 'tournsize_survival': 5, 'ngen': 30, 'verbose': 1}. Best is trial 0 with value: 157.69462365591482.


...

30 	250   	189.849 	262.667	61.0997	277.867
gen	nevals	Mean    	Max 	Std    	max_hof
0  	50    	-72.4573	25.8	28.4324	25.8   
1  	250   	16.4876 	72.6	40.9522	74.6   
2  	250   	64.076  	104.6	13.8351	104.6  
3  	250   	67.236  	124.6	16.6962	124.6  
4  	250   	78.404  	104.6	10.3504	180.8  
5  	250   	90.052  	175.2	32.5745	180.8  
6  	250   	76.976  	125.2	16.6668	180.8  
7  	250   	91.08   	126.6	13.1205	180.8  
8  	250   	96.9213 	169.067	20.6889	180.8  
9  	250   	86.8049 	125.2  	21.5805	180.8  
10 	250   	96.616  	124.6  	16.8177	180.8  
11 	250   	101.424 	198.4  	23.4936	198.4  
12 	250   	121.004 	196    	30.8974	198.4  
13 	250   	101.636 	125.2  	19.6356	198.4  
14 	250   	106.98  	188    	23.458 	198.4  
15 	250   	112.512 	183.2  	18.9267	198.4  
16 	250   	109.732 	181.6  	22.5233	198.4  
17 	250   	111.884 	180    	17.9726	198.4  
18 	250   	103.92  	124.6  	14.9243	198.4  
19 	250   	106.988 	124.6  	15.0247	212    
20 	250   	106.801 	179.467	22.9077	212    
21 	250  

[I 2023-10-15 10:43:10,714] Trial 58 finished with value: 154.22544802867438 and parameters: {'train_enemies': [2, 5, 6], 'initial_range': 3, 'pop_size': 50, 'n_offspring': 5, 'tournsize_parent': 8, 'cxpb': 0.2, 'cx_type': 'whole_aritmetic', 'alpha': 0.30000000000000004, 'mutpb': 1.0, 'mut_type': 'normal', 'sigma': 2.0, 'indpb': 0.5, 'tau': 0.8, 'tournsize_survival': 9, 'ngen': 30, 'verbose': 1}. Best is trial 43 with value: 181.1501792114702.


30 	250   	125.276 	201.333 	33.762 	219.2   
gen	nevals	Mean    	Max     	Std   	max_hof 
0  	50    	-68.9018	-1.75556	26.528	-1.75556
1  	232   	-5.69089	67      	25.8989	67      
2  	230   	39.4424 	94.6    	39.7889	94.6    
3  	228   	64.9524 	185.6   	35.1676	185.6   
4  	237   	86.6867 	185.6   	40.4409	185.6   
5  	239   	80.9747 	185.6   	67.8251	185.6   
6  	219   	123.845 	185.6   	82.9512	185.6   
7  	236   	102.537 	185.6   	78.1118	185.6   
8  	234   	102.316 	185.6   	86.9392	185.6   
9  	229   	133.969 	185.6   	72.5253	185.6   
10 	232   	103.12  	185.6   	90.4857	185.6   
11 	226   	145.501 	185.6   	70.8689	185.6   
12 	241   	84.1282 	185.6   	85.4732	185.6   
13 	240   	102.828 	185.6   	82.2227	185.6   
14 	234   	118.876 	198.667 	85.2674	198.667 
15 	240   	90.7558 	198.667 	80.1459	198.667 
16 	227   	148.709 	198.667 	63.8875	198.667 
17 	234   	112.028 	221.6   	72.9679	221.6   
18 	232   	134.6   	221.6   	69.5805	221.6   
19 	242   	112.126 	221.6   	69.3623

Process ForkPoolWorker-716:
Process ForkPoolWorker-714:
Process ForkPoolWorker-715:
Process ForkPoolWorker-713:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 108, in run
    self

KeyboardInterrupt: 

In [ ]:
#Trial 43 finished with value: 181.1501792114702 and parameters: {'train_enemies': [2, 5, 6], 'initial_range': 3, 'pop_size': 50, 'n_offspring': 5, 'tournsize_parent': 10, 'cxpb': 0.2, 'cx_type': 'whole_aritmetic', 'alpha': 0.6, 'mutpb': 1.0, 'mut_type': 'normal', 'sigma': 1.5, 'indpb': 0.5, 'tau': 0.9, 'tournsize_survival': 8, 'ngen': 30, 'verbose': 1}

In [12]:
res_pd = pd.DataFrame(hof_tuning).sort_values('max_value',ascending=False).reset_index()

In [14]:
res_pd.to_csv("v3_v2_res.csv")

In [ ]:
"""
params = {'initial_range':,
          'pop_size':,
          'n_offspring':,
          'tournsize_parent':,
          'cxpb':,
          'cx_type':,
          'alpha':,
          'mutpb':,
          'mut_type':,
          'sigma':,
          'indpb':,
          'tau':,
          'tournsize_survival':,
          'ngen':,
          'verbose':}
""""